Input: sequence library, query sequence, pdb
Output: sequence library w/ labeled columns

Method 1: Load sequence library from database

Method 2: Load sequence from PDB (make datastructure so pdb is easy to edit?)

Method 3: Align query sequence with pdb sequence

Method 4: Replace temperature with desired value in PDB

In [ ]:
# wt seq aligned with dataset
wt_oh_gaps = input_wt.reshape(1, *input_wt.size())
wt_seq_gaps = utils.one_hots_to_proteins(wt_oh_gaps)
wt_divs_gaps = torch.cat(list(divs_wt.values()))
labels = {'free':slice(0,1), 'fixed':slice(1,2), 'one_free':slice(2,17), 'one_fixed':slice(17,32)}

# wt seq with no gaps
wt_seq_divs = utils.remove_gaps(wt_seq_gaps, wt_divs_gaps)
wt_seq = wt_seq_divs[0]
wt_divs = wt_seq_divs[1:]

# wt seq aligned with 3vdh
wt_seq_align = np.array(wt_aln) # taken from cell 3
wt_divs_align = np.full((len(wt_divs), len(wt_seq_align)), np.nan)
base_i = 0
for (align_i, aa) in enumerate(wt_seq_align):
    if aa == '-':
        pass
    elif wt_seq[base_i] == aa:
        wt_divs_align[:, align_i] = wt_divs[:, base_i]
        base_i += 1
    else:
        print('uh oh the sequences don\'t match!')
        print(base_i, align_i)
        print(wt_seq[base_i], aa)
        break
i = None

# final product: wt_divs_align

In [ ]:
pdblines = open(pdb_path).read().strip().split('\n')

# weird is the index for 'chain identifier'
# idk what this means but sometimes an atom will repeat itself
# and this seems related to it.
atom = slice(13,16)
weird = slice(16,17)
aa = slice(17,20)
dimer = slice(21, 22)
resi = slice(22,26)
temp = slice(60,66)

everything_else = [l for l in pdblines if not
                   (len(l)>30 and l[0:4]=='ATOM' and (l[dimer]=='A' or l[dimer]=='B') and not l[weird]=='B')]

d_a = [l for l in pdblines if len(l)>30 and l[0:4]=='ATOM' and l[dimer]=='A' and not l[weird]=='B']
d_b = [l for l in pdblines if len(l)>30 and l[0:4]=='ATOM' and l[dimer]=='B' and not l[weird]=='B']
d_b = d_b[5:]

# Both start at residue 7
divs_3vdh = wt_divs_align

In [ ]:
# coloring for full variation


for div_type in range(32):
    pdb_a = ''
    pdb_b = ''
    for i, (line_a, line_b) in enumerate(zip(d_a, d_b)):
        res_num = int(line_a[resi])
        div = divs_3vdh[div_type][res_num]
        if np.isnan(div) or div < 0:
            div = 0
        div_formatted =('%0.2f'%(div)).rjust(5)

        pdb_a += line_a.replace(line_a[temp], div_formatted) + '\n'
        pdb_b += line_b.replace(line_b[temp], div_formatted) + '\n'

    new_pdb = pdb_a + '\n' + pdb_b
    open('4im4_div_%d.pdb'%div_type, 'w').write(new_pdb)

In [ ]:
# coloring for loopless to check if it's actually loopless

loopless_path = '../sequence_sets/3vdh_wt_aligned.fasta'
seqs = list(SeqIO.parse(loopless_path, 'fasta'))
wt_seq = seqs[0].seq
is_loop = [int(not aa == '-') for aa in wt_seq]

pdb_path = '../sequence_sets/3vdh.pdb'
pdblines = open(pdb_path).read().strip().split('\n')

loc = {'name': slice(0, 4), 'atom':slice(13,16), 'weird': slice(16, 17), 'aa': slice(17, 20),
       'dimer': slice(21, 22), 'resi': slice(22, 26), 'temp': slice(60, 66)}
d_a = [l for l in Apdblines if len(l)>30 and l[loc['name']]=='ATOM' and l[loc['dimer']]=='A' and not l[loc['weird']]=='B']
d_b = [l for l in pdblines if len(l)>30 and l[loc['name']]=='ATOM' and l[loc['dimer']]=='B' and not l[loc['weird']]=='B']

# aligns d_a and d_b, since it seems that d_b is a few residues longer. . .
while not d_a[0][loc['resi']] == d_b[0][loc['resi']]:
    if d_a[0][loc['resi']] < d_b[0][loc['resi']]:
        del(d_a[0])
    else:
        del(d_b[0])

pdb_a = ''
pdb_b = ''
for i, (line_a, line_b) in enumerate(zip(d_a, d_b)):
    res_num = int(line_a[loc['resi']])
    val = is_loop[res_num]
    val_formatted =('%0.2f'%(val)).rjust(5)

    pdb_a += line_a.replace(line_a[loc['temp']], val_formatted) + '\n'
    pdb_b += line_b.replace(line_b[loc['temp']], val_formatted) + '\n'
    
new_pdb = pdb_a + '\n' + pdb_b
open('loop_check.pdb', 'w').write(new_pdb)